# SEAL-ImplementingSealInferenceInput

- Checking SEAL input compatiblity with previous ones

In [28]:
import jsonlines
import json
from pprint import pprint
import torch
import numpy as np
import random
from copy import deepcopy
from util import utils
from abc import ABC, abstractmethod

In [29]:
def open_jsonl(file):
    data = []
    with open(file, "r") as fin:
        for k, example in enumerate(fin):
            example = json.loads(example)
            data.append(example)
    return data

In [30]:
def check_status(instances):
    print(len(instances))
    print(instances[-1].keys())

## DPR NQ retrieved 
    - dev_file : /data/philhoon-relevance/FiD/open_domain_data/NQ/dev.json
    - test_file : /data/philhoon-relevance/FiD/open_domain_data/NQ/test.json

In [67]:
fid_nq_dev_file = '/data/philhoon-relevance/FiD/open_domain_data/NQ/dev.json'
fid_nq_test_file = '/data/philhoon-relevance/FiD/open_domain_data/NQ/test.json'

In [ ]:
# fid_nq_dpr_dev = utils.open_json(fid_nq_dev_file)
# fid_nq_dpr_test = utils.open_json(fid_nq_test_file)

## contriever NQ  retrieved
    - dev_file : /data/philhoon-relevance/contriever/NQ/contriever-msmarco/dev.json
    - test_file : /data/philhoon-relevance/contriever/NQ/contriever-msmarco/test.json

In [78]:
# contriever_nq_dev_file = '/data/philhoon-relevance/contriever/NQ/contriever-msmarco/dev.jsonl'
# contriever_nq_test_file = '/data/philhoon-relevance/contriever/NQ/contriever-msmarco/test.jsonl'

In [79]:
# contriever_nq_dev = open_jsonl(contriever_nq_dev_file)
# contriever_nq_test = open_jsonl(contriever_nq_test_file)

In [80]:
# check_status(contriever_nq_dev)

In [81]:
# check_status(contriever_nq_test)

In [16]:
# pprint(contriever_nq_test[0])

## SEAL input Format
    - Since we have DPR NQ retrieved format, going to use those files
    
    - dev_file : /data/philhoon-relevance/FiD/open_domain_data/NQ/dev.json
    - test_file : /data/philhoon-relevance/FiD/open_domain_data/NQ/test.json

```python
class DprQueryQasIterator(QueryIterator):

    def get_query(self, id_):
        return self.topics[id_]['question']

    @classmethod
    def from_topics(cls, topics_path: str):
        topics = {}
        order = []
        with open(topics_path) as fin:
            fin = csv.reader(fin, delimiter="\t", quotechar='"')
            for id_, (query, answers) in enumerate(fin):
                answers = ast.literal_eval(answers)
                assert isinstance(answers, list) and isinstance(answers[0], str)
                topics[id_] = {
                    "question": query,
                    "answers": answers,
                }
                order.append(id_)
        return cls(topics, order)
```

In [68]:
class QueryIterator(ABC):

    PREDEFINED_ORDER = {'msmarco-doc-dev',
                        'msmarco-doc-test',
                        'msmarco-passage-dev-subset',
                        'msmarco-passage-test-subset'}

    def __init__(self, topics: dict, order: list = None):
        self.order = order if order else sorted(topics.keys())
        self.topics = topics

    @abstractmethod
    def get_query(self, id_):
        raise NotImplementedError()

    @classmethod
    @abstractmethod
    def from_topics(cls, topics_path: str):
        raise NotImplementedError()

    def __iter__(self):
        for id_ in self.order:
            yield id_, self.get_query(id_)

    def __len__(self):
        return len(self.topics.keys())

    @staticmethod
    def get_predefined_order(topics_path: str):
        order = None
        normalized_path = Path(topics_path).stem  # get filename w/o extension
        normalized_path = normalized_path.replace('_', '-')

        if normalized_path in QueryIterator.PREDEFINED_ORDER:
            print(f'Using pre-defined topic order for {normalized_path}')
            # Lazy import:
            from pyserini.query_iterator_order_info import QUERY_IDS
            order = QUERY_IDS[topics_path]
        return order

In [69]:
from enum import unique, Enum

In [70]:
@unique
class TopicsFormat(Enum):
    DEFAULT = 'default'
    KILT = 'kilt'
    KILT_TEMPLATE = 'kilt_template'
    DPR = 'dpr'
    DPR_OUT = 'dpr_out'
    DPR_QAS = 'dpr_qas'
    NQ = 'nq'

In [75]:
topics_format = TopicsFormat('dpr_out')

In [73]:
# def get_query_iterator(topics_path: str, topics_format: TopicsFormat, queries_path: Optional[str] = None):
#     mapping = {
#         TopicsFormat.DEFAULT: DefaultQueryIterator,
#         TopicsFormat.KILT: KiltQueryIterator,
#         TopicsFormat.KILT_TEMPLATE: KiltTemplateQueryIterator,
#         TopicsFormat.DPR: DprQueryIterator,
#         TopicsFormat.DPR_QAS: DprQueryQasIterator,
#         TopicsFormat.NQ: NqQueryIterator,
#     }
#     return mapping[topics_format].from_topics(topics_path)

In [74]:
# mapping = {
#         TopicsFormat.DEFAULT: DefaultQueryIterator,
#         TopicsFormat.KILT: KiltQueryIterator,
#         TopicsFormat.KILT_TEMPLATE: KiltTemplateQueryIterator,
#         TopicsFormat.DPR: DprQueryIterator,
#         TopicsFormat.DPR_QAS: DprQueryQasIterator,
#         TopicsFormat.NQ: NqQueryIterator,
# }

In [76]:
topics_format

<TopicsFormat.DPR_OUT: 'dpr_out'>

In [66]:
# DprQueryQasIterator(input)

In [98]:
class DprOutputFormatQueryQasIterator(QueryIterator):

    def get_query(self, id_):
        return self.topics[id_]['question']

    @classmethod
    def from_topics(cls, topics_path: str):
        topics = {}
        order = []
        
        with open(topics_path , 'r') as f: 
            data = json.load(f)
        
        for id_, instance in enumerate(data):
            query = instance['question']
            answers = instance['answers']
            assert isinstance(answers, list) and isinstance(answers[0], str)
            topics[id_] = {
                "question": query,
                "answers": answers,
            }
            order.append(id_)
        return cls(topics, order)

In [99]:
dpr_nq_dev_ = DprOutputFormatQueryQasIterator.from_topics(fid_nq_dev_file)

In [103]:
dpr_nq_dev_.order

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [ ]:
# topics = {}
# order = []
# with open(topics_path) as fin:
#     for id_, instance in enumerate(fin):
#         example = json.loads(instance)
#         query = example['question']
#         answers = example['answers']
#         assert isinstance(answers, list) and isinstance(answers[0], str)
#         topics[id_] = {
#             "question": query,
#             "answers": answers,
#         }
#         order.append(id_)
# return cls(topics, order)

In [82]:
# def open_json(file):
#     with open(file , 'r') as f: 
#         data = json.load(f)
#     return data

In [93]:
# def get_instance(file):
#     topics = {}
#     order = []

#     with open(file , 'r') as f: 
#         data = json.load(f)

#     for id_, instance in enumerate(fid_nq_dpr_dev):
#         query = instance['question']
#         answers = instance['answers']
#         assert isinstance(answers, list) and isinstance(answers[0], str)
#         topics[id_] = {
#             "question": query,
#             "answers": answers,
#         }
#         order.append(id_)
#     return topics, order

In [97]:
# instance

In [94]:
# check = get_instance(fid_nq_dev_file)

In [104]:
# check[0]

In [65]:
# a.topics